In [174]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)
import numpy as np

sp500_history = pd.read_csv('./sp500_history.csv')
sp500_history['date'] = pd.to_datetime(sp500_history['date']) 
sp500_history = sp500_history[['date', 'value', 'variable']]

# start_date = datetime.strptime('2019-1-1', '%Y-%m-%d')
# end_date = datetime.strptime('2019-12-31', '%Y-%m-%d')

# # greater than the start date and smaller than the end date
# mask = (sp500_history['date'] > start_date) & (sp500_history['date'] <= end_date)
# sp500_history = sp500_history.loc[mask]

# Sort
sp500_history.sort_values(['date', 'variable'], ascending=[True, True])

,date,value,variable
0,1983-11-30,STE,added_ticker
1,1983-11-30,RHI,added_ticker
2,1983-11-30,AON,added_ticker
3,1983-11-30,SCHW,added_ticker
4,1983-11-30,AMGN,added_ticker
...,...,...,...
955,2019-12-23,TRIP,removed_ticker
959,2020-01-28,PAYC,added_ticker
958,2020-01-28,WCG,removed_ticker
960,2020-03-03,GDI,added_ticker


In [165]:
columns_to_create = ['total', 'mean', 'std', 'up', 'count', 'max', 'min', 'sharpe', 'uppct']

# output = pd.DataFrame(columns=columns_to_create)
# output.set_index(['date', 'add/delete'])

output_add = pd.DataFrame(columns=columns_to_create)
output_delete = pd.DataFrame(columns=columns_to_create)
# output_add['date'] = pd.to_datetime(output_add['date'])
# output_add.set_index('date')
# output_delete['date'] = pd.to_datetime(output_delete['date'])
# output_delete.set_index('date')

In [166]:
MAX_DATE_EXTENSION = 10
days_bef_aft = 10
last_date = sp500_history.iloc[0, sp500_history.columns.get_loc('date')]
last_type = sp500_history.iloc[0, sp500_history.columns.get_loc('variable')]
last_changes = []
is_begin = True

for idx, row in sp500_history.iterrows():

    # End date is exclusive, so need to increase by 1
    prices = pdr.get_data_yahoo(row['value'], start=row['date'] - timedelta(days=days_bef_aft), end=row['date'] + timedelta(days=days_bef_aft + 1))
    if len(prices) == 0:
        continue
        
    # Find the previous trading day
    prev_date = row['date'] - timedelta(days=1)
    date_extension = 0
    while not prev_date in prices.index and date_extension < MAX_DATE_EXTENSION:
        prev_date -= timedelta(days=1)
        date_extension += 1
    if date_extension >= MAX_DATE_EXTENSION:
        continue
    prev_close = prices.loc[prev_date]['Close']

    # Find the next opening price
    next_date = row['date']
    date_extension = 0
    while not next_date in prices.index and date_extension < MAX_DATE_EXTENSION:
        next_date += timedelta(days=1)
        date_extension += 1
    if date_extension >= MAX_DATE_EXTENSION:
        continue    
    next_open = prices.loc[next_date]['Open']

    price_change = next_open - prev_close

    # Calculate mean and all
    if not is_begin and (last_date != row['date'] or last_type != row['variable']):
        print(last_changes)
        count = len(last_changes)
        mean = last_row['total'] / count
        std = np.std(last_changes)
        up = len(list(filter(lambda x: (x >= 0), last_changes))) 
        max_val = max(last_changes)
        min_val = min(last_changes)
        if std == 0:
            sharpe = 0
        else:
            sharpe = mean / std * np.sqrt(252)
        uppct = up / count

#             data = {'mean': mean, 'std': std, 'up': up, 'count': count, 'max': max_val, 'min': min_val, 'sharpe': sharpe, 'uppct': uppct}
        data = [mean, std, up, count, max_val, min_val, sharpe, uppct]

        if last_type == 'added_ticker':
            output_add.loc[last_date, 'mean':] = data

        else:
            output_delete.loc[last_date, 'mean':] = data

        total = 0
        last_changes = []
    
    # Still in the same date
    else:
        if is_begin:
            total = 0
            is_begin = False
        else:
            total = last_row['total']
    
    total += price_change

    if row['variable'] == 'added_ticker':
        if row['date'] not in output_add.index:
            data = {'total': total}
            output_add = output_add.append(pd.DataFrame(data, index = [pd.to_datetime(row['date'])]), ignore_index=False)
        else: 
            output_add.loc[row['date'], 'total'] = total

    else:
        if row['date'] not in output_delete.index:
            data = {'total': total}
            output_delete = output_delete.append(pd.DataFrame(data, index = [pd.to_datetime(row['date'])]), ignore_index=False)
        else: 
            output_delete.loc[row['date'], 'total'] = total

    last_date = row['date']
    last_type = row['variable']
    last_changes.append(price_change)
    
    if last_type == 'added_ticker':
        last_row = output_add.loc[last_date, :]
    else:
        last_row = output_delete.loc[last_date, :]

output_add.index.names = ['date']
output_delete.index.names = ['date']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[-0.3600006103515625]
[*********************100%***********************]  1 of 1 completed
[0.0]
[*********************100%***********************]  1 of 1 completed
[0.2999997138977051]
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[2.4300079345703125]
[*********************100%***********************]  1 of 1 completed
[0.279998779296875]
[*********************100%***********************]  1 of 1 completed
[0.03999900817871094]
[*********************100%***********************]  1 of 1 completed
[-0.48999786376953125]
[*********************100%***********************]  1 of 1 completed
[1.6199989318847656]
[*********************100%***********************]  1 of 1 completed
[-0.29000091552734375]
[*********************100%***********************]  1 of 1 com

In [173]:
output_add.to_csv('sp500_add.csv')
output_delete.to_csv('sp500_delete.csv')